In [1]:
import numpy as np
import dm4bem
import pandas as pd
import matplotlib as plt
from modélisationV5 import*
from GG import*
#%%
#INPUTS

C = np.zeros(A.shape[1])
C[4]=C_thermal['mur haut béton']
C[6]=C_thermal['mur haut isolant']
C[11]=C_thermal['mur haut béton']
C[13]=C_thermal['mur haut isolant']
C[18]=C_thermal['mur intérieur vertical']
C[24]=C_thermal['mur intérieur horizontal']
C[21]=C_thermal['mur intérieur horizontal']
C[29]=C_thermal['mur bas béton']
C[27]=C_thermal['mur bas isolant']

pièce 0 :  26.399002352921862 , pièce 1 :  26.399002352921844 , pièce 2 :  26.372165707021313


In [31]:
# # State-space
# temperature nodes
theta_list=[f"theta_{i}" for i in range(ntheta)]
q_list=[f"q_{i}" for i in range(nbranches)]
# flow-rate branches

#print(help(dm4bem.tc2ss))
Adf = pd.DataFrame(A, index=q_list, columns=theta_list)
Cdf= pd.Series(C, index=theta_list)
Gdf = pd.Series(G, index=q_list)
bdf = pd.Series(b, index=q_list)
fdf = pd.Series(f, index=theta_list)
ydf= pd.Series(y, index=theta_list)

TC = {'A':Adf, 'C':Cdf, 'G':Gdf, 'b':bdf, 'f':fdf, 'y':ydf}
[As, Bs, Cs, Ds, us] = dm4bem.tc2ss(TC)
us

q_0         20.0
q_1         20.0
q_7         20.0
q_35        20.0
q_37        20.0
q_39        20.0
q_40        20.0
theta_3     50.0
theta_7     50.0
theta_8     50.0
theta_10    50.0
theta_14    50.0
theta_16    50.0
theta_17    50.0
theta_19    50.0
theta_20    50.0
theta_22    50.0
theta_23    50.0
theta_25    50.0
theta_26    50.0
theta_30    50.0
theta_32    50.0
dtype: float64

In [12]:
Qa=0 # verif si c'est pas nul
Qin = 50
Qout = 50


         # vector of nonzero elements of vector f : [f0, f3, f7, f8, f10, f14, f16, f17, f19, f20, f22, f23, f25, f26, f30, f32]
#%%
lamda = np.linalg.eig(As)[0] # eigenvalues
dtmax = 2*min(-1/lamda)
dt = dm4bem.round_time(dtmax)

t_settle=4*max(-1/lamda)
duration=np.ceil(t_settle/3600)*3600
n = int(np.floor(duration / dt))    # number of time steps
To=15
Ti_sp=19
V_To = To * np.ones(n)        # outdoor temperature
V_Ti_sp = Ti_sp * np.ones(n)
bT = np.array([To, To, To, To, To, To, Ti_sp]) # vector of the nonzero elements of vector b : temperatures for the conductances 0, 1, 7, 35, 37, 39 and 40.

fQ = np.array([Qa, Qout, Qin, Qout, Qout, Qin, Qout, Qin, Qin, Qin, Qin, Qin, Qin, Qin, Qout, Qout])


In [22]:
uss = np.hstack([bT, fQ])           # input vector for state space
print(f'us = {us}')


us = q_0         20.0
q_1         20.0
q_7         20.0
q_35        20.0
q_37        20.0
q_39        20.0
q_40        20.0
theta_3     50.0
theta_7     50.0
theta_8     50.0
theta_10    50.0
theta_14    50.0
theta_16    50.0
theta_17    50.0
theta_19    50.0
theta_20    50.0
theta_22    50.0
theta_23    50.0
theta_25    50.0
theta_26    50.0
theta_30    50.0
theta_32    50.0
dtype: float64


In [35]:
# Create input_data_set
# ---------------------
# time vector
n = int(np.floor(duration / dt))    # number of time steps
# DateTimeIndex starting at "00:00:00" with a time step of dt
time = pd.date_range(start="2000-01-01 00:00:00",
                           periods=n, freq=f"{int(dt)}S")


phi0=0





f0 = Qa * np.ones(n)       # auxiliary heat gains
f3 = Qout * np.ones(n)      # solar radiation absorbed by the outdoor part of the glasses and the walls
f7 = Qin * np.ones(n)       # solar radiation absorbed by the outdoor part of the glasses and the walls
Φa=Qa=Φi=Φo=0*np.ones(n)
data = {'To': To, 'Ti_sp': Ti_sp, 'Φo': Φo, 'Φi': Φi, 'Qa': Qa, 'Φa': Φa}
input_data_set = pd.DataFrame(data, index=time)
us




q_0         20.0
q_1         20.0
q_7         20.0
q_35        20.0
q_37        20.0
q_39        20.0
q_40        20.0
theta_3     50.0
theta_7     50.0
theta_8     50.0
theta_10    50.0
theta_14    50.0
theta_16    50.0
theta_17    50.0
theta_19    50.0
theta_20    50.0
theta_22    50.0
theta_23    50.0
theta_25    50.0
theta_26    50.0
theta_30    50.0
theta_32    50.0
dtype: float64

In [34]:
# inputs in time from input_data_set
u = dm4bem.inputs_in_time(us, input_data_set)


KeyError: 20.0